### The data has already been generated in the multi_mackend_madel_data folder. You can choose not to run this code file as the random data in it may not be reproducible

In [ ]:
import json, os, pickle, random
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
import qiskit
from qiskit import QuantumCircuit, execute
from qiskit.compiler import transpile
from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.circuit.library import CXGate, RXGate, IGate, ZGate
from qiskit.providers.fake_provider import FakeMontreal, FakeLima,FakeGuadalupe,FakeSherbrooke,FakePrague,FakeCairo
from blackwater.data.utils import get_backend_properties_v1
from qiskit.circuit.random import random_circuit
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# 设置理想和含噪声的后端参数
from qiskit.providers.fake_provider import FakeGuadalupe,FakeMontreal,FakeCairo,FakeMumbai,FakeSydney,FakeToronto
backend_list = []
properties_dict = {}
run_config_ideal_list = []
run_config_noisy_list = []
backend = FakeGuadalupe()
backend_list.append(backend)
backend = FakeMontreal()
backend_list.append(backend)
backend = FakeCairo()
backend_list.append(backend)
backend = FakeMumbai()
backend_list.append(backend)
backend = FakeSydney()
backend_list.append(backend)
backend = FakeToronto()
backend_list.append(backend)
for backend in backend_list:
    properties = get_backend_properties_v1(backend)
    backend_name = backend.name()
    properties_dict[backend_name] = properties
    backend_ideal = QasmSimulator() # Noiseless
    backend_noisy = AerSimulator.from_backend(backend)
    shots = 100000
    run_config_ideal = {'shots': shots, 'backend': backend_ideal, 'name': 'ideal'}
    run_config_ideal_list.append(run_config_ideal)
    run_config_noisy = {'shots': shots, 'backend': backend_noisy, 'name': 'noisy'}
    run_config_noisy_list.append(run_config_noisy)

In [ ]:
#  Train Data
N_FILES = 5
N_ENTRIES_PER_FILE = 80
for  backend_index,backend in enumerate(backend_list):
    backend_name = backend.name()
    for i in tqdm_notebook(range(N_FILES)):
        entries = []
        for index_i in range(N_ENTRIES_PER_FILE):
            ciucuit_depth = random.randint(5, 10)
            circuit = transpile(
                    random_circuit(i+5, ciucuit_depth, measure=True),
                    backend,
                    optimization_level=3,
                )
            while(len(circuit)<10):
                circuit = transpile(
                    random_circuit(i+5, ciucuit_depth, measure=True),
                    backend,
                    optimization_level=3,
                )
            # circuit = transpiled_ansatz.bind_parameters(np.random.uniform(-5, 5, (num_params)))
            qasm_circuit = circuit.qasm().strip()
            with open(f"./multi_backend_model_data/train_qasm/{backend_name}_{i+5}_qubits_{index_i}.qasm",'w') as file:
                file.write(qasm_circuit)
            job_ideal = execute(circuit, **run_config_ideal_list[backend_index])
            job_noisy_1 = execute(circuit, **run_config_noisy_list[backend_index])
            job_noisy_2 = execute(circuit, **run_config_noisy_list[backend_index])
            job_noisy_3 = execute(circuit, **run_config_noisy_list[backend_index])
            counts_ideal = dict(job_ideal.result().get_counts()) # dict
            counts_noisy_fake_1 = dict(job_noisy_1.result().get_counts()) # dict
            counts_noisy_fake_2 = dict(job_noisy_2.result().get_counts()) # dict
            counts_noisy_fake_3 = dict(job_noisy_3.result().get_counts()) # dict
            to_append = {}
            to_append['circuit'] = circuit
            to_append['ideal_exp_value'] = counts_ideal
            to_append['noisy_exp_values_1'] = counts_noisy_fake_1
            to_append['noisy_exp_values_2'] = counts_noisy_fake_2
            to_append['noisy_exp_values_3'] = counts_noisy_fake_3
            entries.append(to_append)
        with open(f"./multi_backend_model_data/train_data/{backend_name}_{i+5}_qubit.pk", "wb") as pk_file:
            pickle.dump(entries, pk_file)
            

In [ ]:
# Test Data
N_FILES = 2
N_ENTRIES_PER_FILE = 20
for  backend_index,backend in enumerate(backend_list):
    backend_name = backend.name()
    for i in tqdm_notebook(range(N_FILES)):
        entries = []
        for index_i in range(N_ENTRIES_PER_FILE):
            ciucuit_depth = random.randint(1, 10)
            circuit = transpile(
                    random_circuit(i+2, ciucuit_depth, measure=True),
                    backend,
                    optimization_level=3,
                )
            while(len(circuit)<10):
                circuit = transpile(
                    random_circuit(i+2, ciucuit_depth, measure=True),
                    backend,
                    optimization_level=3,
                )
            # circuit = transpiled_ansatz.bind_parameters(np.random.uniform(-5, 5, (num_params)))
            qasm_circuit = circuit.qasm().strip()
            with open(f"./multi_backend_model_data/test_qasm/{backend_name}_{i+2}_qubits_{index_i}.qasm",'w') as file:
                file.write(qasm_circuit)
            job_ideal = execute(circuit, **run_config_ideal_list[backend_index])
            job_noisy_1 = execute(circuit, **run_config_noisy_list[backend_index])
            job_noisy_2 = execute(circuit, **run_config_noisy_list[backend_index])
            job_noisy_3 = execute(circuit, **run_config_noisy_list[backend_index])
            counts_ideal = dict(job_ideal.result().get_counts()) # dict
            counts_noisy_fake_1 = dict(job_noisy_1.result().get_counts()) # dict
            counts_noisy_fake_2 = dict(job_noisy_2.result().get_counts()) # dict
            counts_noisy_fake_3 = dict(job_noisy_3.result().get_counts()) # dict
            to_append = {}
            to_append['circuit'] = circuit
            to_append['ideal_exp_value'] = counts_ideal
            to_append['noisy_exp_values_1'] = counts_noisy_fake_1
            to_append['noisy_exp_values_2'] = counts_noisy_fake_2
            to_append['noisy_exp_values_3'] = counts_noisy_fake_3
            entries.append(to_append)
        with open(f"./multi_backend_model_data/test_data/{backend_name}_{i+2}_qubit.pk", "wb") as pk_file:
            pickle.dump(entries, pk_file)

In [ ]:
for  backend_index,backend in enumerate(backend_list):
    backend_name = backend.name()
    for qubits in tqdm_notebook(range(13,14)):
        entries = []
        for MQT_name in MQT_test_list:
            qasm_path= f'./MQT_Bench/{MQT_name}_{str(qubits)}.qasm'
            try:
                circuit = QuantumCircuit.from_qasm_file(qasm_path)
            except:
                continue
            circuit = transpile(
                circuit,
                backend,
                optimization_level=3
            )
            job_ideal = execute(circuit, **run_config_ideal_list[backend_index])
            job_noisy_1 = execute(circuit, **run_config_noisy_list[backend_index])
            job_noisy_2 = execute(circuit, **run_config_noisy_list[backend_index])
            job_noisy_3 = execute(circuit, **run_config_noisy_list[backend_index])
            counts_ideal = dict(job_ideal.result().get_counts()) # dict
            counts_noisy_fake_1 = dict(job_noisy_1.result().get_counts()) # dict
            counts_noisy_fake_2 = dict(job_noisy_2.result().get_counts()) # dict
            counts_noisy_fake_3 = dict(job_noisy_3.result().get_counts()) # dict
            to_append = {}
            to_append['circuit'] = circuit
            to_append['ideal_exp_value'] = counts_ideal
            to_append['noisy_exp_values_1'] = counts_noisy_fake_1
            to_append['noisy_exp_values_2'] = counts_noisy_fake_2
            to_append['noisy_exp_values_3'] = counts_noisy_fake_3
            entries.append(to_append)
        with open(f"./multi_backend_model_data/MQT_test/{backend_name}_{qubits}_qubit.pk", "wb") as pk_file:
                pickle.dump(entries, pk_file)
        